In [0]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential

## 1. Download the ​mini-flowers​ dataset. This contains 1,500 images of five different types of flowers. Modify one of the above notebooks to classify these images using transfer learning. How accurate of a model can you train?

I tried transfer learning from VGG16. I tried two different models: one with 128-node dense layer at the end and the other one with 64 layers. They both have a very similar accuracy of less than 85% accuracy. I also tried other variants of layers, which are not recorded in this notebook, (I only my highest accuracy results), but the highest accruacy I could get was 83%. 

Sources: https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_solution.ipynb#scrollTo=wX9ISOABudWh

In [2]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE

def read_tfrecord(example):
    features = {
        "image": tf.FixedLenFeature((), tf.string), # tf.string means byte string
        "label": tf.FixedLenFeature((), tf.string),
        "one_hot_label": tf.FixedLenFeature((), tf.string)
    }
    example = tf.parse_single_example(example, features)
    image = tf.image.decode_jpeg(example['image'])
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    one_hot_label = tf.io.decode_raw(example['one_hot_label'], out_type=tf.uint8) # 'decode' byte string into byte list
    one_hot_label = tf.cast(one_hot_label, tf.float32)  # convert one hot labels to floats
    one_hot_label = tf.reshape(one_hot_label, [5])  # explicit fixed size needed on TPU
    label = example['label']  # byte string
    return image, label, one_hot_label
  
def load_dataset(filenames):  
  # read from tfrecs
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=32)  # this will read from multiple GCS files in parallel
  dataset = records.map(read_tfrecord, num_parallel_calls=32)
  return dataset

def features_and_targets(image, label, one_hot_label):
  feature = image
  target = one_hot_label
  return feature, target  # for training, a Keras model needs 2 items: features and targets

def get_batched_dataset(filenames):
  dataset = load_dataset(filenames)
  dataset = dataset.map(features_and_targets, num_parallel_calls=32)
  dataset = dataset.cache()  # This dataset fits in RAM
  dataset = dataset.repeat()
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder needed on TPU
  dataset = dataset.prefetch(-1) # prefetch next batch while training  (-1: autotune prefetch buffer size)
  # should shuffle too but this dataset was well shuffled on disk already
  return dataset

def get_training_dataset():
  return get_batched_dataset(training_filenames)

def get_validation_dataset():
  return get_batched_dataset(validation_filenames)

conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files
Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step
Instructions for updating:
Use tf.cast instead.
Epoch 1/8
93/93 [==============================] - 34s 370ms/step - loss: 1.0068 - acc: 0.6522 - val_loss: 0.6078 - val_acc: 0.7842
Epoch 2/8
93/93 [==============================] - 26s 275ms/step - loss: 0.4317 - acc: 0.8558 - val_loss: 0.6105 - val_acc: 0.7783
Epoch 3/8
93/93 [==============================] - 26s 274ms/step - loss: 0.2768 - acc: 0.9210 - val_loss: 0.5351 - val_acc: 0.8006
Epoch 4/8
93/93 [==============================] - 25s 273ms/step - loss: 0.1915 - acc: 0.9499 - val_loss: 0.5109 - val_acc: 0.7991
Epoch 5/8
93/93 [==============================] - 26s 275ms/step - loss: 0.1179 - acc: 0.9772 - val_loss: 0.5043 - val_acc: 0.8274
Epoch 6/8
93/93 [==============================] - 25s 274ms

## 2 Next, run experiments using at least two pretrained convolutional bases (​applications​), and compare your results. Include a short, informal write-up (using bullet points is fine). What differences do you see, and why? Read the associated papers to learn more about the networks you’re using, linked from the API doc.

I have tried VGG16, ResNet, and MobileNet. VGG16.

VGG16: 80% accuracy

ResNet: 52% accruacy

MobileNet: 80% accuracy

Out of all of those models, MobileNet was fastest. I belive that's because the model is built to utilize limited resources with fast performance. 

VGG16 and MobileNet's accuracy came out to be the same and ResNet's accuracy was the loewst. I believe the reason ResNet's accruacy came out so low is because its dataset that the model's trained on is not as similar with flowers datasets as VGG16 or that of MobileNet. 


In [3]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE

conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files
Epoch 1/8
93/93 [==============================] - 29s 309ms/step - loss: 0.9188 - acc: 0.6996 - val_loss: 0.7746 - val_acc: 0.7560
Epoch 2/8
93/93 [==============================] - 25s 274ms/step - loss: 0.3724 - acc: 0.8730 - val_loss: 0.5839 - val_acc: 0.7932
Epoch 3/8
93/93 [==============================] - 26s 275ms/step - loss: 0.2305 - acc: 0.9220 - val_loss: 0.9141 - val_acc: 0.7277
Epoch 4/8
93/93 [==============================] - 25s 274ms/step - loss: 0.1818 - acc: 0.9395 - val_loss: 0.6581 - val_acc: 0.7872
Epoch 5/8
93/93 [==============================] - 26s 276ms/step - loss: 0.1698 - acc: 0.9365 - val_loss: 0.6440 - val_acc: 0.7857
Epoch 6/8
93/93 [==============================] - 26s 276ms/step - loss: 0.0831 - acc: 0.9735 - val_loss: 0.5355 - val_acc: 0.8304
Epoch 7/8
93/93 [==============================] - 26s 276ms/step - loss: 0.0497 - acc: 0.9886 - val_loss: 0.5730

In [4]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE


conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Epoch 1/8
93/93 [==============================] - 28s 306ms/step - loss: 5.9984 - acc: 0.5998 - val_loss: 10.5950 - val_acc: 0.2396
Epoch 2/8
93/93 [==============================] - 23s 249ms/step - loss: 5.2276 - acc: 0.6623 - val_loss: 11.2420 - val_acc: 0.1905
Epoch 3/8
93/93 [==============================] - 23s 248ms/step - loss: 4.8178 - acc: 0.6922 - val_loss: 12.2829 - val_acc: 0.2366
Epoch 4/8
93/93 [==============================] - 23s 248ms/step - loss: 4.4570 - acc: 0.7174 - val_loss: 11.4012 - val_acc: 0.2426
Epoch 5/8
93/93 [==============================] - 23s 249ms/step - loss: 4.1370 - acc: 0.7372 - val_loss: 9.9512 - val_acc: 0.3631
Epoch 6/8
93/93 [==============================] - 23s 249ms/step - loss: 4.2906 - acc: 0.7285 - val_loss: 9.0679 - val_acc: 0.4182
Epoch 7/8
93/93 [==============================] - 23s 248ms/step - loss: 4.2094 - acc: 0.7335 - val_loss: 7.4845 - val_acc: 0.5208
Epoch 8

In [5]:
BATCH_SIZE = 32
EPOCHS = 8
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-192x192/*.tfrec'
IMAGE_SIZE = [192, 192]
VALIDATION_SPLIT = 0.19
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips'] # do not change, maps to the labels in the data (folder names)
filenames = tf.gfile.Glob(GCS_PATTERN)
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
validation_filenames = filenames[:split]
print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files".format(len(filenames), len(training_filenames), len(validation_filenames)))
validation_steps = int(3670 // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(3670 // len(filenames) * len(training_filenames)) // BATCH_SIZE


conv_base = MobileNet(weights='imagenet', include_top=False, input_shape=(192, 192, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

conv_base.trainable = False

model.compile(
  optimizer=tf.train.AdamOptimizer(),
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

history = model.fit(get_training_dataset(), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                      validation_data=get_validation_dataset(), validation_steps=validation_steps)

Pattern matches 16 data files. Splitting dataset into 13 training files and 3 validation files
17227776/17225924 [==============================] - 0s 0us/step
Epoch 1/8
93/93 [==============================] - 12s 130ms/step - loss: 7.3270 - acc: 0.5198 - val_loss: 6.7170 - val_acc: 0.5759
Epoch 2/8
93/93 [==============================] - 8s 83ms/step - loss: 5.6711 - acc: 0.6361 - val_loss: 4.5909 - val_acc: 0.7039
Epoch 3/8
93/93 [==============================] - 8s 84ms/step - loss: 4.4345 - acc: 0.7127 - val_loss: 4.6243 - val_acc: 0.7039
Epoch 4/8
93/93 [==============================] - 8s 84ms/step - loss: 3.8218 - acc: 0.7483 - val_loss: 3.1555 - val_acc: 0.7902
Epoch 5/8
93/93 [==============================] - 8s 83ms/step - loss: 2.2782 - acc: 0.8471 - val_loss: 2.7400 - val_acc: 0.8185
Epoch 6/8
93/93 [==============================] - 8s 83ms/step - loss: 2.3884 - acc: 0.8401 - val_loss: 2.5092 - val_acc: 0.8289
Epoch 7/8
93/93 [==============================] - 8s 83ms